# Mobile breadboard!




<img align="left" src="images/mine/tank_front.jpg" width=900>


<img align="right" src="images/mine/tank_diag.jpg" width=400>

- Lolin32 Lite Devboard
- Led lit NDC front
- max7219 8x8 LED Matrix display
- DHT22 temperature and humidity sensor
- 2x Piezo buzzers
- <font color="brown">Cat</font>


## LED Matrix

<img src="images/components/max7219 led matrix.jpg" width=400>

In [1]:
print(1)

1


In [32]:
import max7219
import time
from machine import Pin, SPI
spi = SPI(-1, 10000000, miso=Pin(19), mosi=Pin(23), sck=Pin(18))
display = max7219.Matrix8x8(spi, Pin(17), 4)

In [33]:
display.brightness(15)
display.fill(0)
display.text('1234',0,0,1)
display.show()

In [38]:
from time import sleep

with open('ledtext.txt', 'r') as f:
    for l in f:
        print(l)
        display.fill(0)
        display.text(l,0,0,1)
        display.show()
        sleep(1)

display.fill(0)
display.show()


----

ALL 

HAIL

THE 

DEMO

GODS


## Temperature and humidity

<img src="images/components/dht22.jpg" width=400>

DHT22
- 1-wire interface (newer have i2c)

MicroPython [dht](https://docs.micropython.org/en/latest/esp8266/esp8266/tutorial/dht.html) module

In [12]:
import dht
import machine
d = dht.DHT22(machine.Pin(4))

In [13]:
d.measure()
t = d.temperature()
h = d.humidity()
print("{} degrees and {}% humidity".format(t, h))

24.1 degrees and 49.2% humidity


In [14]:
display.fill(0)
display.text('{}C'.format(int(t)),0,0,1)
display.show()

## Mobile breadboard

In [24]:
to_left = Pin(12, Pin.OUT)

In [26]:
to_left.value(0)

## Pulse-width modulation (PWM)

<img align="right" src="images/concepts/Pwm_5steps.gif" width=600>

- Frequency [Hz]
- Duty [%]

<img src="images/concepts/Delta_PWM.png" width=900>

In [2]:
from machine import Pin, PWM
ndc_led = PWM(Pin(27, Pin.OUT), freq=50, duty=0)
to_left = PWM(Pin(12, Pin.OUT), freq=50, duty=0)
to_right = PWM(Pin(14, Pin.OUT), freq=50, duty=0)

In [ ]:
to_left.duty()

In [ ]:
to_left.duty(0)

## Uasyncio

In [3]:
import max7219
import uasyncio
from machine import Pin, SPI, PWM

spi = SPI(-1, 10000000, miso=Pin(19), mosi=Pin(23), sck=Pin(18))

display = max7219.Matrix8x8(spi, Pin(17), 4)
display.brightness(15)

buz_ch1 = PWM(Pin(19), freq=440, duty=0)
buz_ch2 = PWM(Pin(32), freq=440, duty=0)
pwm_duty = 400

async def file_to_led():
    while True:
        with open('ledtext.txt', 'r') as f:
            for l in f:
                print(l)
                display.fill(0)
                display.text(l,0,0,1)
                display.show()
                await uasyncio.sleep(0.5)

        display.fill(0)
        display.show()
        await uasyncio.sleep(1)

        
async def temp():
    import dht
    from time import sleep
    
    d = dht.DHT22(Pin(4))
    while True:
        print('measuring..')
        d.measure()
        #await uasyncio.sleep_ms(200)
        t = d.temperature()
        h = d.humidity()
        display.fill(0)
        display.text('{}C'.format(int(t)),0,0,1)
        display.show()
        buz_ch1.duty(0)
        sleep(1)
        await uasyncio.sleep(5)

        
async def play_tone(freq, msec, buz_ch):
    print('{} - freq = {:6.1f} msec = {:6.1f}'.format(buz_ch, freq, msec))
    if freq > 0:
        buz_ch.duty(pwm_duty)
        buz_ch.freq(int(freq))
        await uasyncio.sleep_ms(int(msec * 0.9))
    buz_ch.duty(0)
    await uasyncio.sleep_ms(int(msec * 0.1))


async def play(tune, buz_ch):
    try:
        for freq, msec in tune.notes():
            await play_tone(freq, msec, buz_ch)
    except KeyboardInterrupt:
        await play_tone(0, 0, buz_ch)        
    
    
async def killer():
    print("sleeping {} seconds".format(20))
    await uasyncio.sleep(20)

loop = uasyncio.get_event_loop()
loop.create_task(file_to_led())
loop.create_task(temp())

from rtttl import RTTTL
import songs
loop.create_task(play(RTTTL(songs.find('MissionImp')), buz_ch1))
loop.create_task(play(RTTTL(songs.find('MissionImp')), buz_ch2))

try:
    loop.run_until_complete(killer())
finally:
    buz_ch1.deinit()
    buz_ch2.deinit()
    display.fill(0)
    display.show()

----

measuring..
PWM(19, freq=440, duty=0) - freq = 1174.8 msec =   78.9
PWM(32, freq=1174, duty=0) - freq = 1174.8 msec =   78.9
sleeping 20 seconds
ALL 

PWM(19, freq=1174, duty=0) - freq = 1244.4 msec =   78.9
PWM(32, freq=1244, duty=0) - freq = 1244.4 msec =   78.9
PWM(19, freq=1244, duty=0) - freq = 1174.8 msec =   78.9
PWM(32, freq=1174, duty=0) - freq = 1174.8 msec =   78.9
PWM(19, freq=1174, duty=0) - freq = 1244.4 msec =   78.9
PWM(32, freq=1244, duty=0) - freq = 1244.4 msec =   78.9
PWM(19, freq=1244, duty=0) - freq = 1174.8 msec =   78.9
PWM(32, freq=1174, duty=0) - freq = 1174.8 msec =   78.9
PWM(19, freq=1174, duty=0) - freq = 1244.4 msec =   78.9
PWM(32, freq=1244, duty=0) - freq = 1244.4 msec =   78.9
PWM(19, freq=1244, duty=0) - freq = 1174.8 msec =   78.9
HAIL

PWM(32, freq=1174, duty=0) - freq = 1174.8 msec =   78.9
PWM(19, freq=1174, duty=0) - freq = 1244.4 msec =   78.9
PWM(32, freq=1244, duty=0) - freq = 1244.4 msec =   78.9
PWM(19, freq=1244, duty=0) - freq = 117

## HTTP Webserver

In [ ]:
import machine
sound = machine.PWM(machine.Pin(19), freq=50, duty=0)
DUTY = 0

html = """<!DOCTYPE html>
<html>
    <head> <title>Drive my breadboard!</title> </head>
    <body> <h1>Drive</h1>
        <form>
        <div>
            <button name="left" value="UP" type="submit">Right +</button>
            <button name="left" value="DOWN" type="submit">Left -</button>
        </div>
        <div>
            <button name="right" value="UP" type="submit">Right +</button>
            <button name="right" value="DOWN" type="submit">Right -</button>
        </div>
        </form>
    </body>
</html>
"""

import socket
addr = socket.getaddrinfo('0.0.0.0', 80)[0][-1]

s = socket.socket()
s.bind(addr)
s.listen(1)

print('listening on', addr)

while True:
    cl, addr = s.accept()
    print('client connected from', addr)
    request = cl.recv(1024)
    request = str(request)
    print('Content %s' % request)
    left = request.find('/?turn=LEFT')
    right = request.find('/?turn=RIGHT')
    print(left, right)
    if left == 6:
        DUTY += 1
    if right == 6:
        DUTY -= 1
    DUTY = max(0, min(DUTY, 100))
    sound.duty(DUTY)
    response = html
    cl.send(response)
    cl.close()